In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "trl<0.9.0" xformers

In [2]:
import unsloth
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
max_seq_length = 4096
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen2.5-3B-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True,
)

==((====))==  Unsloth 2025.9.6: Fast Qwen2 patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.05G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Standard rank for conversation patterns
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
)

Unsloth 2025.9.6 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [5]:
from datasets import load_dataset
dataset = load_dataset("bitext/Bitext-customer-support-llm-chatbot-training-dataset", split="train[:3000]")

README.md: 0.00B [00:00, ?B/s]

Bitext_Sample_Customer_Support_Training_(…):   0%|          | 0.00/19.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26872 [00:00<?, ? examples/s]

In [6]:
dataset

Dataset({
    features: ['flags', 'instruction', 'category', 'intent', 'response'],
    num_rows: 3000
})

In [7]:
def format_customer_support(examples):
    """Format examples for customer support conversation task"""
    texts = []

    for i in range(len(examples["instruction"])):
        customer_query = examples["instruction"][i]
        support_response = examples["response"][i]

        # Create customer support conversation format
        messages = [
            {"role": "system", "content": "You are a helpful customer support assistant. Provide accurate, friendly, and professional assistance to customers. Be empathetic and solution-focused."},
            {"role": "user", "content": customer_query},
            {"role": "assistant", "content": support_response}
        ]

        # Apply Qwen2.5 chat template
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False,
        )

        texts.append(text)

    return {"text": texts}

In [8]:
formatted_dataset = dataset.map(
    format_customer_support,
    batched=True,
    remove_columns=dataset.column_names,
    desc="Formatting for customer support conversations"
)

Formatting for customer support conversations:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [9]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,  # Effective batch size: 8
    warmup_steps=10,
    max_steps=120,  # Sufficient for conversation patterns
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=5,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    seed=3407,
    output_dir="qwen25_customer_support",
    save_strategy="no",
)

In [11]:
from trl import SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    args=training_args,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    packing=False,  # Preserve conversation structure
)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [12]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,000 | Num Epochs = 1 | Total steps = 120
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 29,933,568 of 3,115,872,256 (0.96% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abeshith (abeshith-dr-m-g-r-educational-and-research-institute) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
5,3.277900
10,1.981300
15,1.257700
20,1.031500
25,0.873000
30,0.727200
35,0.693900
40,0.658700
45,0.640300
50,0.583200


TrainOutput(global_step=120, training_loss=0.8058204889297486, metrics={'train_runtime': 402.2538, 'train_samples_per_second': 2.387, 'train_steps_per_second': 0.298, 'total_flos': 3472440829820928.0, 'train_loss': 0.8058204889297486, 'epoch': 0.32})

In [13]:
model.save_pretrained("qwen25_support_agent")
tokenizer.save_pretrained("qwen25_support_agent")

('qwen25_support_agent/tokenizer_config.json',
 'qwen25_support_agent/special_tokens_map.json',
 'qwen25_support_agent/chat_template.jinja',
 'qwen25_support_agent/vocab.json',
 'qwen25_support_agent/merges.txt',
 'qwen25_support_agent/added_tokens.json',
 'qwen25_support_agent/tokenizer.json')

In [ ]:
FastLanguageModel.for_inference(model)

In [15]:
def customer_support_chat(query, context="general", max_tokens=400):
    """Handle customer support conversation"""

    system_prompts = {
        "general": "You are a helpful customer support assistant. Provide accurate, friendly, and professional assistance to customers. Be empathetic and solution-focused.",
        "billing": "You are a billing support specialist. Help customers with payment issues, invoices, refunds, and subscription management. Be clear about policies and next steps.",
        "technical": "You are a technical support expert. Help customers troubleshoot issues, provide step-by-step solutions, and escalate complex problems when needed.",
        "sales": "You are a sales support representative. Help customers with product information, pricing, comparisons, and purchase decisions. Be helpful but not pushy.",
        "returns": "You are a returns and exchange specialist. Help customers with return policies, refund processes, and product exchanges. Be understanding and efficient."
    }

    messages = [
        {"role": "system", "content": system_prompts.get(context, system_prompts["general"])},
        {"role": "user", "content": query}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    inputs = tokenizer([text], return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=0.7,  # Balanced for friendly but consistent responses
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
        )

    # Extract generated tokens and decode
    generated_tokens = outputs[0][len(inputs['input_ids'][0]):]
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    return response.strip()

In [16]:
general_queries = [
    "Hi, I'm having trouble logging into my account. Can you help me?",
    "What are your business hours?",
    "I need to speak with a manager about my recent experience.",
    "Can you explain your return policy?",
    "I'm a new customer and need help getting started."
]

print("🛟 GENERAL CUSTOMER SUPPORT:")
for i, query in enumerate(general_queries, 1):
    print(f"\n👤 Customer {i}: {query}")
    response = customer_support_chat(query, "general", 300)
    print(f"🤖 Support Agent: {response}")
    print("-" * 60)

🛟 GENERAL CUSTOMER SUPPORT:

👤 Customer 1: Hi, I'm having trouble logging into my account. Can you help me?
🤖 Support Agent: I apologize for the inconvenience you're experiencing while trying to log in to your account. To assist you further, could you please provide me with your login credentials or any error messages you may be encountering? This will help me identify the issue and provide you with the appropriate solution.
------------------------------------------------------------

👤 Customer 2: What are your business hours?
🤖 Support Agent: Our business hours are Monday through Friday from 9 AM to 5 PM Eastern Time. We're closed on weekends and holidays.
------------------------------------------------------------

👤 Customer 3: I need to speak with a manager about my recent experience.
🤖 Support Agent: I've been informed that you would like to speak with a manager regarding your recent experience. I apologize for any inconvenience this may have caused you. To assist you further, 

In [17]:
billing_queries = [
    "I was charged twice for my subscription this month. Can you help me get a refund?",
    "My payment method was declined but I know I have funds. What should I do?",
    "I want to cancel my subscription. How do I do that?",
    "Can you explain the charges on my last invoice?",
    "I need to update my billing address."
]

print("\n💳 BILLING SUPPORT:")
for i, query in enumerate(billing_queries, 1):
    print(f"\n👤 Customer {i}: {query}")
    response = customer_support_chat(query, "billing", 350)
    print(f"🤖 Billing Support: {response}")
    print("-" * 60)


💳 BILLING SUPPORT:

👤 Customer 1: I was charged twice for my subscription this month. Can you help me get a refund?
🤖 Billing Support: I apologize for the inconvenience caused by the double charge for your subscription. To resolve this issue, I recommend reaching out to our customer support team for assistance. They will be able to investigate the situation and process the necessary refund for you. Please provide them with your order details and any relevant information so they can assist you promptly.
------------------------------------------------------------

👤 Customer 2: My payment method was declined but I know I have funds. What should I do?
🤖 Billing Support: I'm sorry to hear that your payment method was declined despite having the funds available. To resolve this issue, please follow these steps:

1. Check Your Account: Log in to your account on our website and review your payment history. This will help you identify any errors or discrepancies.
2. Contact Support: Reach ou

In [18]:
technical_queries = [
    "The app keeps crashing when I try to upload files. Can you help?",
    "I'm getting an error message that says 'Connection timeout'. What does this mean?",
    "How do I reset my password? The reset email isn't coming through.",
    "The website is loading very slowly on my computer. Is there a problem?",
    "I can't find the feature you mentioned in the tutorial. Where is it located?"
]

print("\n🔧 TECHNICAL SUPPORT:")
for i, query in enumerate(technical_queries, 1):
    print(f"\n👤 Customer {i}: {query}")
    response = customer_support_chat(query, "technical", 400)
    print(f"🤖 Tech Support: {response}")
    print("-" * 60)



🔧 TECHNICAL SUPPORT:

👤 Customer 1: The app keeps crashing when I try to upload files. Can you help?
🤖 Tech Support: I'm sorry to hear that the app is crashing while trying to upload files. To better assist you, could you please provide more information about the issue? Specifically, what do you see on the screen when the app crashes? Are there any error messages or specific steps that you can reproduce? This will help us identify the root cause of the problem and provide the most effective solution.
------------------------------------------------------------

👤 Customer 2: I'm getting an error message that says 'Connection timeout'. What does this mean?
🤖 Tech Support: The error message "Connection timeout" indicates that the system is unable to establish a connection with the required service or server within the specified time frame. This could be due to various reasons such as network connectivity issues, firewall settings, or server downtime. To resolve this issue, you can try t

In [19]:
sales_queries = [
    "What's the difference between your Basic and Premium plans?",
    "Do you offer any discounts for students or nonprofits?",
    "I'm interested in your product but need to know if it integrates with Salesforce.",
    "What's included in the free trial?",
    "Can I upgrade my plan in the middle of my billing cycle?"
]

print("\n🛒 SALES SUPPORT:")
for i, query in enumerate(sales_queries, 1):
    print(f"\n👤 Customer {i}: {query}")
    response = customer_support_chat(query, "sales", 350)
    print(f"🤖 Sales Support: {response}")
    print("-" * 60)


🛒 SALES SUPPORT:

👤 Customer 1: What's the difference between your Basic and Premium plans?
🤖 Sales Support: Our Basic plan offers a limited set of features and benefits, while our Premium plan includes additional features and benefits. The specific differences depend on the type of service you're using. For example, in the context of our product, the Basic plan may include basic features such as access to the platform, while the Premium plan may include advanced features like priority customer support and extended service hours. We recommend contacting our customer support team for more detailed information about the differences between the two plans.
------------------------------------------------------------

👤 Customer 2: Do you offer any discounts for students or nonprofits?
🤖 Sales Support: We do offer discounts for students and nonprofits! We believe in providing value to everyone, so we've created special offers just for you. To take advantage of these discounts, please provi

In [20]:
returns_queries = [
    "I received the wrong item in my order. How can I exchange it?",
    "This product doesn't work as advertised. I'd like to return it.",
    "What's your return policy for items bought on sale?",
    "I need to return something but I threw away the packaging. Is that okay?",
    "How long do I have to return an item?"
]

print("\n📦 RETURNS & EXCHANGES:")
for i, query in enumerate(returns_queries, 1):
    print(f"\n👤 Customer {i}: {query}")
    response = customer_support_chat(query, "returns", 300)
    print(f"🤖 Returns Specialist: {response}")
    print("-" * 60)


📦 RETURNS & EXCHANGES:

👤 Customer 1: I received the wrong item in my order. How can I exchange it?
🤖 Returns Specialist: We understand that you have received the wrong item in your order, and we apologize for any inconvenience this may have caused. To facilitate an exchange, please reach out to our customer support team. They will guide you through the process and ensure that your request is handled promptly and efficiently. Thank you for choosing us, and we appreciate your patience and understanding.
------------------------------------------------------------

👤 Customer 2: This product doesn't work as advertised. I'd like to return it.
🤖 Returns Specialist: We appreciate your feedback and the need for us to assist you with the return process for this product that doesn't meet your expectations. To ensure a smooth experience, please follow these steps:

1. Contact our customer support team during business hours (X:XX - X:XX) at {{Customer Support Phone Number}} or use the Live Chat

In [21]:
complex_queries = [
    "I've been trying to use your service for a week but keep running into problems. First, my account wasn't activated properly, then I couldn't access the main features, and now I'm getting billed for something I never signed up for. This is very frustrating and I need someone to help me sort this out completely.",

    "I'm a business owner looking to switch from your competitor. I need to know: 1) Can you migrate our data? 2) What's the onboarding process like? 3) Do you provide training for my team? 4) What kind of support do we get after implementation?",

    "I bought your premium subscription last month, but I'm not satisfied with the performance. The app is slow, some features don't work, and customer service hasn't been helpful. I want either a full refund or a significant discount, plus assurance that these issues will be fixed."
]

print("\n🔥 COMPLEX CUSTOMER ISSUES:")
for i, query in enumerate(complex_queries, 1):
    print(f"\n👤 Frustrated Customer {i}: {query[:100]}...")
    response = customer_support_chat(query, "general", 500)
    print(f"🤖 Senior Support Agent: {response}")
    print("-" * 60)



🔥 COMPLEX CUSTOMER ISSUES:

👤 Frustrated Customer 1: I've been trying to use your service for a week but keep running into problems. First, my account wa...
🤖 Senior Support Agent: I apologize for any frustration you're experiencing while using our service. It's important that we resolve these issues as soon as possible. Let's start with the first problem: your account wasn't activated properly. We can take care of that by reaching out to our customer support team and providing them with your details. They will be able to assist you in activating your account correctly.

Next, let's address the issue with accessing the main features. Could you please provide us with more information about the specific problems you're facing? This will help us identify the root cause and find a solution to ensure a smooth experience for you.

Lastly, regarding the billing issue, we apologize for any confusion or surprise caused by the charges you didn't sign up for. Our finance team is available to rev

In [22]:
conversation_turns = [
    "Hello, I need help with my recent order.",
    "My order number is #12345. It was supposed to arrive yesterday but I haven't received it.",
    "I checked the tracking and it says 'delivered' but there's nothing at my door.",
    "Yes, I've checked with my neighbors and building management. No one has seen the package.",
    "Okay, I'll file a claim. How long does that usually take?"
]

print("💬 Simulating Multi-Turn Customer Conversation:")
for i, turn in enumerate(conversation_turns, 1):
    print(f"\n👤 Customer Turn {i}: {turn}")
    response = customer_support_chat(turn, "general", 250)
    print(f"🤖 Support Agent: {response}")
    print("-" * 40)

💬 Simulating Multi-Turn Customer Conversation:

👤 Customer Turn 1: Hello, I need help with my recent order.
🤖 Support Agent: I'm here to assist you! Please provide me with more details about the issue you're facing with your recent order so that I can provide the most appropriate guidance.
----------------------------------------

👤 Customer Turn 2: My order number is #12345. It was supposed to arrive yesterday but I haven't received it.
🤖 Support Agent: I'm sorry to hear that you haven't received your order with the order number #12345 as expected. Our team is here to assist you in resolving this issue. To better understand the situation, could you please provide more details about what happened? This will help us take the necessary steps to ensure your order arrives on time.
----------------------------------------

👤 Customer Turn 3: I checked the tracking and it says 'delivered' but there's nothing at my door.
🤖 Support Agent: We apologize for the confusion you may have experienced